In [ ]:
import os
from skimage import io
from glob import glob
from shutil import copyfile
import imageio

import utils.preprocessing

## Data Preparation

In [ ]:
# Mrthod to create an output folder splitting the ground truth in 3 subsets: train, validation and tetst
# Instance labels are transformed into semantic labels, adding an esge class around the objects

def create_train_val_test_split(root, out_folder):
    image_paths = glob(os.path.join(root, "images", "*.tif"))
    image_paths.sort()
    label_paths = glob(os.path.join(root, "labels", "*.tif"))
    label_paths.sort()

    assert len(image_paths) == len(label_paths)

    # we have 40 images with annotations and use 32 for train, 4 for val and 4 for test
    n_train = 32
    n_val = 4

    for ii, (im, lbl) in enumerate(zip(image_paths, label_paths)):
        if ii < n_train:
            split = "train"
        elif ii < (n_train + n_val):
            split = "val"
        else:
            split = "test"
        im_out = os.path.join(out_folder, split, "images")
        lbl_out = os.path.join(out_folder, split, "labels")
        os.makedirs(im_out, exist_ok=True)
        os.makedirs(lbl_out, exist_ok=True)
        copyfile(im, os.path.join(im_out, os.path.split(im)[1]))
        
        # Label pre-processing: instances --> semantic labels (object / edges)
        lbl_image = io.imread(lbl) # import labels mask
        lbl_sem = utils.preprocessing.semantic_edges(lbl_image, 2) # generate semantic labels
        print('Saving:', os.path.join(lbl_out, os.path.split(lbl)[1]))
        imageio.imwrite(os.path.join(lbl_out, os.path.split(lbl)[1]), lbl_sem) # export semantic labels


In [ ]:
root = r'C:\Users\Pau\Desktop\Neurosphere Annotations\export'
out_folder = os.path.join(root, 'out_data_preparation')

create_train_val_test_split(root, out_folder)